In [92]:
import pandas as pd
import numpy as np

In [93]:
df = pd.read_csv('tmdb_movies_data.csv')


In [94]:
df = df[['id','original_title','cast','director','keywords','overview','genres','release_year','vote_average']]

In [95]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)

In [96]:
df.drop(df[df['original_title'] == 'Cazuza - O tempo nÃ£o pÃ¡ra'].index, inplace= True)
df.drop(df[df['original_title'] == 'í•˜ìš¸ë§'].index, inplace= True)
df = df.reset_index(drop=True)

In [97]:
def remove(obj):
    return obj.split('|')

In [98]:
df['cast'] = df['cast'].apply(lambda x:remove(str(x)))

In [99]:
df['keywords'] = df['keywords'].apply(lambda x:remove(str(x)))
df['genres'] = df['genres'].apply(lambda x:remove(str(x)))


In [100]:
rec_df = df

In [101]:
df[df['id'] == 355338]

,id,original_title,cast,director,keywords,overview,genres,release_year,vote_average
251,355338,Riley's First Date?,"[Amy Poehler, Phyllis Smith, Bill Hader, Lewis...",Josh Cooley,"[mother daughter relationship, rock music, gir...","Riley, now 12, is hanging out at home with her...","[Animation, Family]",2015,7.3


In [102]:
rec_df['genres'] = rec_df['genres'].apply(lambda x:x[0])

In [103]:
rec_df = rec_df[['id','original_title','genres','vote_average']]

In [104]:
import pickle
pickle.dump(rec_df,open('movie_data.sav','wb'))

In [105]:
df['cast'] = df['cast'].apply(lambda x:x[:4])


In [106]:
df['cast'] = df['cast'].apply(lambda x:[str(i).replace(' ','') for i in x])

In [107]:
df['director'] = df['director'].apply(lambda x:[str(x).replace(' ','')])

In [108]:
df['keywords'] = df['keywords'].apply(lambda x:[str(i).replace(' ','') for i in x])

In [109]:
df['genres'] = df['genres'].apply(lambda x:[str(i).replace(' ','') for i in x])

In [110]:
df['overview'] = df['overview'].apply(lambda x:str(x).split(' '))

In [111]:
df['release_year'] = df['release_year'].apply(lambda x:[str(x)])

In [112]:
df['corpus'] = df['overview'] + df['keywords'] + df['genres'] + df['cast'] + df['director'] + df['release_year']

In [113]:
new_df = df[['id','original_title','corpus','vote_average']]

In [114]:
new_df['corpus'] = new_df['corpus'].apply(lambda x: " ".join(x))

/tmp/ipykernel_12075/3919970262.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['corpus'] = new_df['corpus'].apply(lambda x: " ".join(x))


In [115]:
new_df['corpus'] = new_df['corpus'].apply(lambda x: str.lower(x))

/tmp/ipykernel_12075/420603864.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['corpus'] = new_df['corpus'].apply(lambda x: str.lower(x))


In [116]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [117]:
def stem(text):
    words = []
    for i in text.split():
        words.append(ps.stem(i))
    
    return " ".join(words)

In [118]:
# new_df['corpus'] = new_df['corpus'].apply(stem)

In [119]:
from sklearn.feature_extraction.text import CountVectorizer

# Stop Words = a,the,of,after,now

In [120]:
cv = CountVectorizer(stop_words='english',min_df=2)

In [121]:
data = cv.fit_transform(new_df['corpus']).toarray()

In [122]:
pickle.dump(data,open('similarity.sav','wb'))

In [123]:
from sklearn.metrics.pairwise import cosine_similarity

In [124]:
similarity = cosine_similarity(data)

In [125]:
new_df = new_df.rename({'original_title':'title'},axis=1)

In [126]:
sorted(list(enumerate(similarity[0])),reverse=True,key= lambda x:x[1])[1:11]

[(4395, 0.27931019386546413),
 (8695, 0.23911404992940524),
 (8666, 0.22156468376279895),
 (5701, 0.18181818181818185),
 (405, 0.16283473681973243),
 (37, 0.15974461276617435),
 (226, 0.14083575804390608),
 (6635, 0.1371450962647483),
 (7723, 0.13483997249264842),
 (85, 0.13273296830047474)]

In [127]:
df.iloc[0]

id                                                           135397
original_title                                       Jurassic World
cast              [ChrisPratt, BryceDallasHoward, IrrfanKhan, Vi...
director                                           [ColinTrevorrow]
keywords          [monster, dna, tyrannosaurusrex, velociraptor,...
overview          [Twenty-two, years, after, the, events, of, Ju...
genres                                           [A, c, t, i, o, n]
release_year                                                 [2015]
vote_average                                                    6.5
corpus            [Twenty-two, years, after, the, events, of, Ju...
Name: 0, dtype: object

In [128]:
def recommend(movie):
    movie_index = new_df[new_df['id'] == int(movie)].index[0]
    movie_list = sorted(list(enumerate(similarity[movie_index])),reverse=True,key= lambda x:x[1])[1:11]
    rec_movies = []
    for i in movie_list:
        rec_movies.append(new_df.iloc[i[0]][0])
    return {'ids':rec_movies}

In [129]:
new_df[new_df['id'] == 253412].index[0]

52

In [130]:
new_df[new_df['title'] == 'The Dark Knight'].iloc[0][0]

155

In [131]:
recommend(76341)


{'ids': [27786, 9659, 14642, 20504, 38695, 11036, 4917, 310135, 287424, 84226]}

In [132]:
genre = [
    {
        "id": 28,
        "name": "Action"
    },
    {
        "id": 12,
        "name": "Adventure"
    },
    {
        "id": 16,
        "name": "Animation"
    },
    {
        "id": 35,
        "name": "Comedy"
    },
    {
        "id": 80,
        "name": "Crime"
    },
    {
        "id": 99,
        "name": "Documentary"
    },
    {
        "id": 18,
        "name": "Drama"
    },
    {
        "id": 10751,
        "name": "Family"
    },
    {
        "id": 14,
        "name": "Fantasy"
    },
    {
        "id": 36,
        "name": "History"
    },
    {
        "id": 27,
        "name": "Horror"
    },
    {
        "id": 10402,
        "name": "Music"
    },
    {
        "id": 9648,
        "name": "Mystery"
    },
    {
        "id": 10749,
        "name": "Romance"
    },
    {
        "id": 878,
        "name": "Science Fiction"
    },
    {
        "id": 10770,
        "name": "TV Movie"
    },
    {
        "id": 53,
        "name": "Thriller"
    },
    {
        "id": 10752,
        "name": "War"
    },
    {
        "id": 37,
        "name": "Western"
    }
]

In [133]:
import json
movie_lists = []
for i in genre:
    movie_lists.append({ 
        i['name'] :rec_df[rec_df['genres'] == i['name']].iloc[0:19]['id'].to_list()
    })


In [134]:
new_df.reset_index(drop=True)
pickle.dump(new_df,open('data.sav','wb'))
pickle.dump(similarity,open('similarity.sav','wb'))

In [135]:
pickle.dump(similarity,open('similarity.pbz2','wb'))

In [136]:
import random
print(random.randint(0,9306))

2712
